In [38]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import metrics as mt, preprocessing as pp
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import Imputer, OneHotEncoder
#from sklearn.ensemble import RandomForestClassifier

In [34]:
dados = pd.read_csv('datasets/adult.csv')
#dados.count()
dados.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'result']
dados

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,result
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K


In [36]:
#colunas preenchidas e que não precisam ser alteradas:: 0, 2, 4, 12

#coluna 1 contem elementos faltando entao transformar em numeros e preencher com o elemento mais frequente
    #transformar em numero onde '?' é 0.
class_mapping_1 = {label: idx for idx, label in enumerate(np.unique(dados['workclass']))}
dados['workclass'] = dados['workclass'].map(class_mapping_1)

#alterar coluna 3 colocando numeros
class_mapping_3 = {label: idx for idx, label in enumerate(np.unique(dados['education']))}
dados['education'] = dados['education'].map(class_mapping_3)

#alterar coluna 5 colocando numeros
class_mapping_5 = {label: idx for idx, label in enumerate(np.unique(dados['marital-status']))}
dados['marital-status'] = dados['marital-status'].map(class_mapping_5)

#alterar coluna 6 colocando numeros
class_mapping_6 = {label: idx for idx, label in enumerate(np.unique(dados['occupation']))}
dados['occupation'] = dados['occupation'].map(class_mapping_6)

#alterar coluna 7 colocando numeros
class_mapping_7 = {label: idx for idx, label in enumerate(np.unique(dados['relationship']))}
dados['relationship'] = dados['relationship'].map(class_mapping_7)

#alterar coluna 8 em varias colunas e eliminar a coluna 'Other'
class_mapping_8 = {label: idx for idx, label in enumerate(np.unique(dados['race']))}
dados['race'] = dados['race'].map(class_mapping_8)

#alterar coluna 9 para binario:: 1 para homem e 0 para mulher
class_mapping_9 = {label: idx for idx, label in enumerate(np.unique(dados['sex']))}
dados['sex'] = dados['sex'].map(class_mapping_9)

#alterar coluna 13 para numeros (paises)
class_mapping_13 = {label: idx for idx, label in enumerate(np.unique(dados['native-country']))}
dados['native-country'] = dados['native-country'].map(class_mapping_13)

#alterar coluna 14 para binario:: 0 para <=50K e 1 para >50K
class_mapping_14 = {label: idx for idx, label in enumerate(np.unique(dados['result']))}
dados['result'] = dados['result'].map(class_mapping_14)

dados.head()
#Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
#preencher com o numero mais frequente
#imr = Imputer(missing_values=0, strategy='most_frequent', axis=0)
#imr = imr.fit(dados.values)
#dados = imr.transform(dados.values)
#coluna 10 e 11 só ha alguns valores preenchidos: excluir é a melhor opção

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,result
0,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
1,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
2,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
3,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0
4,37,4,284582,12,14,2,4,5,4,0,0,0,40,39,0


# Use 5-Fold Cross Validation Estratificada para obter a Accuracy através do uso dos seguintes algoritmos: Logistic Regression, kNN, Naive Bayes, SVM.

    Faça Standardization dos dados (fit para o conjunto de treino e transform para treino e teste).

Escolha a melhor técnica e crie um modelo final usando todo o dataset para treinar o modelo.

In [19]:
y = dados.iloc[:, 14].values

X = dados.iloc[:, 1:14].values

X,y

(array([[     6,  83311,      9, ...,      0,     13,     39],
        [     4, 215646,     11, ...,      0,     40,     39],
        [     4, 234721,      1, ...,      0,     40,     39],
        ...,
        [     4, 151910,     11, ...,      0,     40,     39],
        [     4, 201490,     11, ...,      0,     20,     39],
        [     5, 287927,     11, ...,      0,     40,     39]]),
 array([0, 0, 0, ..., 0, 0, 1]))

In [21]:
vt_acc_lr  = []
vt_acc_knn = []
vt_acc_nb  = []
vt_acc_svm = []

clf = StratifiedKFold(n_splits=5)

for train_index, test_index in clf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    std = pp.StandardScaler().fit(X_train)
    X_train_std = std.transform(X_train)
    X_test_std = std.transform(X_test)
    
    md_lr = LogisticRegression().fit(X_train_std, y_train)
    y_pred_lr = md_lr.predict(X_test_std)
    vt_acc_lr.append(mt.accuracy_score(y_test, y_pred_lr))
    
    md_knn = LogisticRegression().fit(X_train_std, y_train)
    y_pred_knn = md_knn.predict(X_test_std)
    vt_acc_knn.append(mt.accuracy_score(y_test, y_pred_knn))
    
    md_nb = LogisticRegression().fit(X_train_std, y_train)
    y_pred_nb = md_nb.predict(X_test_std)
    vt_acc_nb.append(mt.accuracy_score(y_test, y_pred_nb))
    
    md_svm = LogisticRegression().fit(X_train_std, y_train)
    y_pred_svm = md_svm.predict(X_test)
    vt_acc_svm.append(mt.accuracy_score(y_test, y_pred_svm))

/home/yaagocruuz/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [22]:
class_names = ['LR', 'kNN','NB','SVM']
class_1 = [vt_acc_lr, vt_acc_knn, vt_acc_nb, vt_acc_svm]

print('Accuracy Logistic Regression = {}'.format(np.mean(vt_acc_lr)))
print('Accuracy kNN = {}'.format(np.mean(vt_acc_knn)))
print('Accuracy Naive Bayes = {}'.format(np.mean(vt_acc_nb)))
print('Accuracy SVM = {}'.format(np.mean(vt_acc_svm)))

Accuracy Logistic Regression = 0.8246622689763132
Accuracy kNN = 0.8246622689763132
Accuracy Naive Bayes = 0.8246622689763132
Accuracy SVM = 0.24081695087261895
